# Install

## Install BenchMARL

In [ ]:
#@title
!git clone https://github.com/facebookresearch/BenchMARL

In [ ]:
#@title
%cd /content/BenchMARL
!pip install -U torch torchvision
!pip install -e .

## Install Envs

In [ ]:
#@title
%cd /content/BenchMARL
!pip install vmas

In [ ]:
%cd /content/BenchMARL
!pip install dm-meltingpot

In [ ]:
%cd /content/BenchMARL
!pip install benchmarl pettingzoo[atari] supersuit wandb

In [ ]:
#@title
!apt-get update
!apt-get install -y x11-utils python3-opengl xvfb
!pip install pyvirtualdisplay
import pyvirtualdisplay
display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
display.start()

# Launch from command line

# VMAS Football

## MAPPO

In [ ]:
!python benchmarl/run.py -m \
  algorithm=mappo \
  task=vmas/football \
  model=layers/mlp \
  critic_model=\${model} \
  seed=0,1 \
  experiment.max_n_frames=245760 \
  experiment.train_device=cuda \
  experiment.sampling_device=cuda \
  experiment.buffer_device=cpu \
  experiment.render=false \
  experiment.on_policy_n_envs_per_worker=2 \
  experiment.on_policy_collected_frames_per_batch=4096 \
  experiment.evaluation_interval=12288 \
  experiment.on_policy_minibatch_size=256 \
  experiment.on_policy_n_minibatch_iters=4 \
  algorithm.entropy_coef=0.01 \
  experiment.lr=3e-4 \
  task.n_blue_agents=2 \
  task.n_red_agents=2 \
  task.ai_red_agents=false \
  task.dense_reward=true \
  task.observe_teammates=true \
  task.observe_adversaries=true \
  "experiment.loggers=[csv]"


## QMIX

In [ ]:
!python benchmarl/run.py -m \
  algorithm=qmix \
  task=vmas/football \
  model=layers/mlp \
  critic_model=\${model} \
  seed=0,1 \
  experiment.max_n_frames=245760 \
  experiment.train_device=cuda \
  experiment.sampling_device=cuda \
  experiment.buffer_device=cpu \
  experiment.render=false \
  experiment.off_policy_n_envs_per_worker=2 \
  experiment.off_policy_collected_frames_per_batch=4096 \
  experiment.evaluation_interval=12288 \
  experiment.off_policy_train_batch_size=256 \
  experiment.off_policy_n_optimizer_steps=4 \
  experiment.off_policy_init_random_frames=0 \
  experiment.lr=3e-4 \
  task.n_blue_agents=2 \
  task.n_red_agents=2 \
  task.ai_red_agents=false \
  task.dense_reward=true \
  task.observe_teammates=true \
  task.observe_adversaries=true \
  "experiment.loggers=[csv]"


[2025-04-14 01:05:46,465][HYDRA] Launching 2 jobs locally
[2025-04-14 01:05:46,465][HYDRA] 	#0 : algorithm=qmix task=vmas/football model=layers/mlp critic_model=$\{model\} seed=0 experiment.max_n_frames=245760 experiment.train_device=cuda experiment.sampling_device=cuda experiment.buffer_device=cpu experiment.render=False experiment.off_policy_n_envs_per_worker=2 experiment.off_policy_collected_frames_per_batch=4096 experiment.evaluation_interval=12288 experiment.off_policy_train_batch_size=256 experiment.off_policy_n_optimizer_steps=4 experiment.off_policy_init_random_frames=0 experiment.lr=0.0003 task.n_blue_agents=2 task.n_red_agents=2 task.ai_red_agents=False task.dense_reward=True task.observe_teammates=True task.observe_adversaries=True experiment.loggers=[csv]

Algorithm: qmix, Task: vmas/football

Loaded config:

experiment:
  sampling_device: cuda
  train_device: cuda
  buffer_device: cpu
  share_policy_params: true
  prefer_continuous_actions: true
  collect_with_grad: false


## MASAC

In [ ]:
!python benchmarl/run.py -m \
  algorithm=masac \
  task=vmas/football \
  model=layers/mlp \
  critic_model=\${model} \
  seed=0,1 \
  experiment.max_n_frames=245760 \
  experiment.train_device=cuda \
  experiment.sampling_device=cuda \
  experiment.buffer_device=cpu \
  experiment.render=false \
  experiment.off_policy_n_envs_per_worker=2 \
  experiment.off_policy_collected_frames_per_batch=4096 \
  experiment.evaluation_interval=12288 \
  experiment.off_policy_train_batch_size=256 \
  experiment.off_policy_n_optimizer_steps=4 \
  experiment.off_policy_init_random_frames=0 \
  experiment.lr=3e-4 \
  task.n_blue_agents=2 \
  task.n_red_agents=2 \
  task.ai_red_agents=false \
  task.dense_reward=true \
  task.observe_teammates=true \
  task.observe_adversaries=true \
  "experiment.loggers=[csv]"



# PettingZoo - Simple Tag

# MAPPO

In [ ]:
!python benchmarl/run.py -m \
  algorithm=mappo \
  task=pettingzoo/simple_tag \
  model=layers/mlp \
  critic_model=\${model} \
  seed=0,1 \
  experiment.max_n_frames=245760 \
  experiment.train_device=cuda \
  experiment.sampling_device=cuda \
  experiment.buffer_device=cpu \
  experiment.render=false \
  experiment.on_policy_n_envs_per_worker=1 \
  experiment.on_policy_collected_frames_per_batch=4096 \
  experiment.on_policy_minibatch_size=256 \
  experiment.on_policy_n_minibatch_iters=4 \
  experiment.evaluation_interval=12288 \
  algorithm.entropy_coef=0.01 \
  experiment.lr=3e-4 \
  "experiment.loggers=[csv]"

## QMIX

In [ ]:
!python benchmarl/run.py -m \
  algorithm=qmix \
  task=pettingzoo/simple_tag \
  model=layers/mlp \
  critic_model=\${model} \
  seed=0,1 \
  experiment.max_n_frames=245760 \
  experiment.train_device=cuda \
  experiment.sampling_device=cuda \
  experiment.buffer_device=cpu \
  experiment.render=false \
  experiment.off_policy_n_envs_per_worker=2 \
  experiment.off_policy_collected_frames_per_batch=4096 \
  experiment.off_policy_train_batch_size=256 \
  experiment.off_policy_n_optimizer_steps=4 \
  experiment.off_policy_init_random_frames=0 \
  experiment.evaluation_interval=12288 \
  experiment.lr=3e-4 \
  "experiment.loggers=[csv]"

# MASAC

In [ ]:
!python benchmarl/run.py -m \
  algorithm=masac \
  task=pettingzoo/simple_tag \
  model=layers/mlp \
  critic_model=\${model} \
  seed=0,1 \
  experiment.max_n_frames=245760 \
  experiment.train_device=cuda \
  experiment.sampling_device=cuda \
  experiment.buffer_device=cpu \
  experiment.render=false \
  experiment.off_policy_n_envs_per_worker=2 \
  experiment.off_policy_collected_frames_per_batch=4096 \
  experiment.off_policy_train_batch_size=256 \
  experiment.off_policy_n_optimizer_steps=4 \
  experiment.off_policy_init_random_frames=0 \
  experiment.evaluation_interval=12288 \
  experiment.lr=3e-4 \
  "experiment.loggers=[csv]"
